In [1]:
import pandas as pd
import scipy.io
import numpy as np
from scipy.spatial.distance import pdist, squareform
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import tensorflow as tf
import time
from scipy.spatial import distance


# Data  

I am using the QM7 dataset, which is a subset of GDB-13 (a database of nearly 1 billion stable and synthetically accessible organic molecules) containing up to 7 heavy atoms C, N, O, and S. The 3D Cartesian coordinates of the most stable conformations and their atomization energies were determined using ab-initio density functional theory (PBE0/tier2 basis set). This dataset also provided Coulomb matrices as calculated in [Rupp et al. PRL, 2012]:  
  * $C_{i,i} = 0.5 \cdot Z^2.4$  
  * $C_{i,j} = Z_i \cdot \frac{Z_j}{|(R_i−R_j)|}$ 
  * $Z_i$ - nuclear charge of atom i  
  * $R_i$ - cartesian coordinates of atom i  

The data file (.mat format, we recommend using `scipy.io.loadmat` for python users) contains five arrays:  
  * "X" - (7165 x 23 x 23), Coulomb matrices  
  * "T" - (7165), atomization energies (unit: kcal/mol)  
  * "P" - (5 x 1433), cross-validation splits as used in [Montavon et al. NIPS, 2012]  
  * "Z" - (7165 x 23), atomic charges  
  * "R" - (7165 x 23 x 3), cartesian coordinate (unit: Bohr) of each atom in the molecules   

# Process Data 

In [2]:
qm7 = scipy.io.loadmat('Data/qm7.mat')
X,T,P,Z,R = qm7['X'], qm7['T'], qm7['P'], qm7['Z'], qm7['R'] 
y = np.transpose(qm7['T']).reshape((7165,))
y = y/2000 # atomization energy
y # Label

array([-0.20898 , -0.35621 , -0.282105, ..., -0.83105 , -0.891005,
       -0.9595  ], dtype=float32)

In [3]:
num_atoms = X.shape[1]
final_data = []
max_len = num_atoms*num_atoms + 2
for id, molecue in enumerate(X):
    lst = []
    dis = []
    
    distance_matrix = distance.cdist(R[id], R[id], 'euclidean')
    for i in range(num_atoms):
        if molecue[i][i] == 0:
            break
        lst.append(molecue[i][i])
    lst.append(0)
    for i in range(num_atoms):
        if molecue[i][i] == 0:
            break 
        for j in range(i+1, num_atoms):
            if molecue[i][j] == 0:
                break
            lst.append(molecue[i][j])
            dis.append(distance_matrix[i][j])
    lt = lst+[0]+dis
    while (len(lt) < max_len):
        lt.append(0)
    final_data.append(lt)
transform_data = np.array(final_data)
transform_data = transform_data/100 

# Train_data

In [4]:
print("Atomization Energy:\n{}".format(y))
print("Molecule Reprsent:\n{}".format(transform_data))

Atomization Energy:
[-0.20898  -0.35621  -0.282105 ... -0.83105  -0.891005 -0.9595  ]
Molecule Reprsent:
[[0.36858105 0.005      0.005      ... 0.         0.         0.        ]
 [0.36858105 0.36858105 0.005      ... 0.         0.         0.        ]
 [0.36858105 0.36858105 0.005      ... 0.         0.         0.        ]
 ...
 [0.36858105 0.53358707 0.36858105 ... 0.         0.         0.        ]
 [0.36858105 0.36858105 0.36858105 ... 0.         0.         0.        ]
 [0.36858105 0.36858105 0.36858105 ... 0.         0.         0.        ]]


# Model Training 

### 1. XGBoost

In [5]:
rand_state = 123

In [6]:
import xgboost as xgb
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE 
from sklearn.metrics import mean_absolute_error as MAE 
from sklearn.model_selection import cross_val_score, KFold

In [20]:
n_folds = P.shape[0]-1
result = []
index = 0
test_X = transform_data[P[index]]
test_y = y[P[index]]
P_train = np.stack(tuple(P[i] for i in range(n_folds+1) if i != index), axis = 0)

model_xgb = xgb.XGBRegressor(objective ='reg:linear', n_estimators = 300, seed = 123, eval_metric = 'rmse', max_depth = 6) 

for id, cross in enumerate(P_train):
    x_val = transform_data[cross]
    y_val = y[cross] 
    train_id = [l for l in range(n_folds) if l != id]
    x_train = np.concatenate(tuple(transform_data[P_train[i]] for i in train_id), axis = 0)
    y_train = np.concatenate(tuple(y[P_train[i]] for i in train_id), axis = 0)
    model_xgb.fit(x_train, y_train) 
    pred = model_xgb.predict(x_val)
    # RMSE Computation 
    rmse = np.sqrt(MSE(y_val, pred)) 
    result.append(rmse)
    
print(result)
print("RMSE_training: {}".format(np.mean(np.array(result))))
pred_test = model_xgb.predict(test_X)
rmse = np.sqrt(MSE(test_y, pred_test)) 
print("RMSE_testing: {}".format(rmse*2000))




/Users/apple/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [15:32:27] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
/Users/apple/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [15:32:33] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
/Users/apple/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [15:32:39] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
/Users/apple/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [15:32:45] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:li

[0.011701697, 0.008478755, 0.012975742, 0.0077675963]
RMSE_training: 0.01023094728589058
RMSE_testing: 16.70544408261776


### 2. SVR

In [19]:
from sklearn.svm import SVR 

n_folds = P.shape[0]-1
result = []
index = 4
test_X = transform_data[P[index]]
test_y = y[P[index]]
P_train = np.stack(tuple(P[i] for i in range(n_folds+1) if i != index), axis = 0)

model_svr = SVR(kernel='linear') 
# poly, rbf 

for id, cross in enumerate(P_train):
    x_val = transform_data[cross]
    y_val = y[cross] 
    train_id = [l for l in range(n_folds) if l != id]
    x_train = np.concatenate(tuple(transform_data[P_train[i]] for i in train_id), axis = 0)
    y_train = np.concatenate(tuple(y[P_train[i]] for i in train_id), axis = 0)
    model_svr.fit(x_train, y_train) 
    pred = model_svr.predict(x_val)
    # RMSE Computation 

    rmse = np.sqrt(MSE(y_val, pred)) 
    result.append(rmse)
    
print(result)
print("RMSE_training: {}".format(np.mean(np.array(result))))
pred_test = model_svr.predict(test_X)
rmse = np.sqrt(MSE(test_y, pred_test)) 
print("RMSE_testing: {}".format(rmse*2000))
MSE(test_y, pred_test)**0.5


[0.04340218450386778, 0.041690432093440914, 0.04078364525696893, 0.04070576945363344]
RMSE_training: 0.04164550782697776
RMSE_testing: 78.95892163733367


0.03947946081866684

## 3. AdaBoost

In [11]:
from sklearn.ensemble import AdaBoostRegressor


n_folds = P.shape[0]-1
result = []
index = 0
test_X = transform_data[P[index]]
test_y = y[P[index]]
P_train = np.stack(tuple(P[i] for i in range(n_folds+1) if i != index), axis = 0)

model_ada = AdaBoostRegressor(random_state=123, n_estimators=100)
# poly, rbf 

for id, cross in enumerate(P_train):
    x_val = transform_data[cross]
    y_val = y[cross] 
    train_id = [l for l in range(n_folds) if l != id]
    x_train = np.concatenate(tuple(transform_data[P_train[i]] for i in train_id), axis = 0)
    y_train = np.concatenate(tuple(y[P_train[i]] for i in train_id), axis = 0)
    model_ada.fit(x_train, y_train) 
    pred = model_ada.predict(x_val)
    # RMSE Computation 

    rmse = np.sqrt(MSE(y_val, pred)) 
    result.append(rmse)
    # break 
    
print(result)
print("RMSE_training: {}".format(np.mean(np.array(result))))
pred_test = model_ada.predict(test_X)
rmse = np.sqrt(MSE(test_y, pred_test)) 
print("RMSE_testing: {}".format(rmse))

[0.025557004455692374, 0.02516945429829853, 0.026004648614615117, 0.024173826339992842]
RMSE_training: 0.025226233427149712
RMSE_testing: 0.024928986078889544


In [18]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import LinearRegression
regr = LinearRegression()

n_folds = P.shape[0]-1
result = []
index = 0
test_X = transform_data[P[index]]
test_y = y[P[index]]
P_train = np.stack(tuple(P[i] for i in range(n_folds+1) if i != index), axis = 0)

model = LinearRegression()
# poly, rbf 

for id, cross in enumerate(P_train):
    x_val = transform_data[cross]
    y_val = y[cross] 
    train_id = [l for l in range(n_folds) if l != id]
    x_train = np.concatenate(tuple(transform_data[P_train[i]] for i in train_id), axis = 0)
    y_train = np.concatenate(tuple(y[P_train[i]] for i in train_id), axis = 0)
    print(x_train.shape, y_train.shape)
    model.fit(x_train, y_train) 
    pred = model.predict(x_val)
    # RMSE Computation 
    # print(pred, y_val)
    rmse = np.sqrt(MSE(y_val, pred)) 
    result.append(rmse)
    break 
    
print(result)
print("RMSE_training: {}".format(np.mean(np.array(result))))
pred_test = model.predict(test_X)
rmse = np.sqrt(MSE(test_y, pred_test)) 
print("RMSE_testing: {}".format(rmse*2000))

(4299, 531) (4299,)
[0.025383040334771585]
RMSE_training: 0.025383040334771585
RMSE_testing: 63.30810352881318
